# Change Detection using SAR

In this notebook, we explore a **method for detecting change over time** that takes advantage of Sentinel-1 SAR data available within the Descartes Labs platform and leverages `Workflows` to create and perform analysis on composites of these data. 

This example demonstrates the detection of **deforested areas** using Sentinel-1 data.  

The cells in this notebook can be run one at a time by using `Shift-Enter`


# Sentinel-1 Log Ratio for Deforestation

Monitoring deforestation is crucial to understanding atmospheric carbon accumulation, biodiversity reduction and other consequences of global forest loss. Nearly half of the world's remaining forests are located in tropical areas; tropical forests also display the highest rate of deforestation [(Keenan et al, 2015)](https://www.sciencedirect.com/science/article/pii/S0378112715003400). Persistent cloud cover in the tropics makes it difficult to use optical sensors to monitor forest cover. For instance, we analyzed the cloud fraction of Sentinel-2 optical images acquired over Borneo in 2018. We found that 5740 (37%) of the 15447 Sentinel-2 scenes acquired during that period had a cloud fraction less than 50%, and only 10% of scenes have a cloud fraction less than 10%. Therefore this example demonstrates how to leverage Synthetic Aperture Radar (SAR) imagery, which is not affected by clouds, to monitor annual forest loss in South East Asia. While we introduce some general principles here, many resources are available to learn more about SAR forestry applications such as this excellent [SAR Handbook](https://servirglobal.net/Global/Articles/Article/2674/sar-handbook-comprehensive-methodologies-for-forest-monitoring-and-biomass-estimation). 

SAR sensors image the world by emitting electromagnetic radiation in the microwave frequencies and quantifying the portion of this signal that is reflected back towards the sensor (backscatter). Forests and dense vegetation in general tend to reflect a large portion of the signal, and therefore appear bright in SAR imagery. Bare earth, on the other hand, reflects very little radiation. Therefore, to identify deforestation we look for pixels that go from high to low brightness in SAR. A simple yet powerful change detection method for SAR data is the log-ratio, which consists of computing the difference in the log space between pairs of SAR images (see for instance [Garzelli & Zopetti 2017](https://ieeexplore.ieee.org/document/8035263)). Thus in this notebook we compute the log-ratio between a 2018 and a 2019 SAR composite over Borneo, in order to identify newly deforested areas in 2019. 

We chose to demonstrate this simple but powerful approach on annual composites to estimate year-over-year forest change, however the 6-12 day revisit rate of Sentinel-1 also allows for monitoring forest at a higher cadence in order to identify deforestation in near-real time. 

## Import packages

In [ ]:
# keep logging quiet
import logging
logging.getLogger().setLevel(logging.INFO)
logging.captureWarnings(True)

In [ ]:
# import packages
import ipywidgets 
import ipyleaflet
import numpy as np
import matplotlib.pyplot as plt
import descarteslabs.workflows as wf
from utils import FullArray, DrawControl

## Sentinel-1 log ratio

Here, we generate Sentinel-1 SAR median composites for 2018 and 2019, and create a log-ratio layer by taking the log of the ratio of these two composites. To generate a deforestation mask, we apply a threshold to the log-ratio layer.  

In [ ]:
# Define & visualize Workflows ImageCollection for before & after Sentinel-1 composites

# 2018 Sentinel-1 composite
vh_2018 = (wf.ImageCollection.from_id("sentinel-1:GRD",
                            start_datetime='2018-01-01',
                            end_datetime='2019-01-01'
                           ).pick_bands(
    "vh").median(axis='images'))
vh_2018.visualize(
    "Sentinel-1 2018 composite", colormap='Greys', scales=[[0, 0.09]])

# 2019 Sentinel-1 composite
vh_2019 = (wf.ImageCollection.from_id("sentinel-1:GRD",
                            start_datetime='2019-01-01',
                            end_datetime='2020-01-01'
                           ).pick_bands(
    "vh").median(axis='images'))          
vh_2019.visualize(
    "Sentinel-1 2019 composite", colormap='Greys', scales=[[0, 0.09]])

# Take the log of the ratio of both composites (equivalent to difference of logs)
log_ratio = wf.log10(vh_2018 / vh_2019)

# Define a threshold for the log ratio
threshold =  0.15

# Threshold the log ratio layer  
deforestation = (log_ratio > threshold) 

# Morphological filtering
The log-ratio methodology is inherently noisy, so here we leverage the Workflows kernel capability to apply some simple morphological filtering to the deforestation layer in order to clean up the deforestation detections.

In [ ]:
# Define simple functions for erosion and dilation

def erode_op(map_layer, iters, kernel):
    map_layer = ~map_layer
    for i in range(iters):
        map_layer = wf.conv2d(map_layer, kernel) > 0
    map_layer = ~map_layer 
    return map_layer

def dilate_op(map_layer, iters, kernel):
    for i in range(iters):
        map_layer = map_layer * 1.0
        map_layer = wf.conv2d(map_layer, kernel) > 0
    return map_layer

# Define a kernel and perform one erosion followed by two dilations
kernel = wf.Kernel(dims=(3,3), data=[0., 1., 0.,
                                      1., 1., 1.,
                                      0., 1., 0.])

eroded = erode_op(deforestation, iters=1, kernel=kernel)
dilated = dilate_op(eroded, iters=2, kernel=kernel)

# Visualize the resulting deforestation mask
lyr = dilated.mask(dilated==0).visualize('Deforestation', checkerboard=False, colormap='bwr')
lyr.opacity = 0.7

## Define custom ipywidget to visualize deforestation mask and calculate deforested acreage

The FullArray call leverages the supporting utilities to enable the user to draw a polygon around an area of interest to output the size of the deforested area. The `print_acreage` function converts this to hectares and displays it next to the map. 

In [ ]:
# Place the center of the map in an interesting area
center = (0.2923, 115.9698)

# Initialize a workflow map
m = wf.map.map
m.center = center
m.zoom = 12
m.layout.width = '1200px'
m.layout.height = '900'

dc = DrawControl(circlemarker={}, rectangle={}, polyline={})
wf.map.add_control(dc)

# Interactively compute deforested acreage 
pixel_sum = FullArray(wf.map, variable=dilated,
                     draw_control=dc)

output = ipywidgets.Output()
@output.capture()
def print_acreage(*args, **kwargs):
    # Estimate the surface area of one pixel in hectares
    pix_size = (156543.00/2**(max(m.zoom, 0)))**2 / 10000
    # Get the array equivalent to the polygon drawn by the user
    d = pixel_sum.data            
    # Compute deforested and total acreage
    deforested = round((d==1).sum() * pix_size, 2)
    total = round(((d==1) | (d==0)).sum() * pix_size, 2)
    print("Deforested area within AOI : \n\n{} ha (of {} ha in total)".format(deforested, total))

# Perform the acreage computation
dc.on_draw(print_acreage)

# Print acreage in a clear button on the map in the top right corner
output_clear_button = ipywidgets.Button(
    description="Clear Output Window", layout=ipywidgets.Layout(width="auto"))
output_clear_button.on_click(lambda b: output.clear_output())
output_vbox = ipywidgets.VBox([output, output_clear_button])
m.add_control(ipyleaflet.WidgetControl(widget=output_vbox, position='bottomleft'))

ipywidgets.HBox([wf.map])